# Scenario C - Peak Number Variation (results evaluation)

This file is used to evaluate the inference (numerical) results.

The model used in the inference of the parameters is formulated as follows: 

\begin{equation}
\large y = f(x) = \sum\limits_{m=1}^M \big[A_m \cdot e^{-\frac{(x-\mu_m)^2}{2\cdot\sigma_m^2}}\big] + \epsilon
\end{equation}

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import arviz as az
import seaborn as sns

#az.style.use('arviz-darkgrid')

print('Running on PyMC3 v{}'.format(pm.__version__))

## Import local modules

In [ ]:
import sys
sys.path.append('../../modules')
import results as res

In [ ]:
def heatmap(data, plist, title, color, fsize, fname="./heatmap", precision=".3f"):
    ''' plots a heatmap from numerical data provided in a NxN matrix '''
    
    sns.set(font_scale=1.3)
    
    plt.figure(figsize=fsize)
    plt.title(title)

    yticks = ["m_%dp" % val for _, val in enumerate(plist)]
    xticks = ["d_%dp" % val for _, val in enumerate(plist)]
    
    sns.heatmap(data, annot=True, fmt=precision, linewidths=.5, linecolor="#DCDCDC", square=True,
                    cmap=color, cbar=False, xticklabels=xticks, yticklabels=yticks)
    #sns.heatmap(data, annot=True, fmt=precision, linewidths=.5, square=True,
    #                cmap=color, cbar=False, xticklabels=xticks, yticklabels=yticks)

    plt.yticks(rotation=0)
    plt.xticks(rotation=0)
    
    plt.savefig(fname + '.png', dpi=150)

## Load results and extract convergence information

In [ ]:
# number of mrun result files
nrun = 2

# load results into list of dataframes
ldf = []
for i in range(nrun):
    fname = './output_mruns_5x5_{0:02d}/scenario_peaks_mruns.csv'.format(i+1)
    print("reading file: {0}".format(fname))
    df = pd.read_csv(fname)
    df.index += 1
    ldf += [df]

In [ ]:
# extract the convergence results per model
peaklist = [2,3,4,5,6]
dres = res.get_model_summary(ldf, peaklist)

In [ ]:
# figure size and color mapping
figs=(7,7)

#coolwarm, *bone, gray, binary, BuPu, YlGn, Blues, *Greens, Purples
col = "Greens"
col_r = col + "_r"

## Heatmaps of n-peak model vs. n-peak number in dataset

### WAIC

In [ ]:
heatmap(dres['waic'], peaklist, "WAIC", col, figs, "hmap_waic", ".0f")

### Rhat

In [ ]:
heatmap(dres['rhat'], peaklist, "RHAT", col, figs, "hmap_rhat")

### R2

In [ ]:
heatmap(dres['r2'], peaklist, "R2", col_r, figs, "hmap_r2")

### BFMI

In [ ]:
heatmap(dres['bfmi'], peaklist, "BFMI", col_r, figs, "hmap_bfmi")

### MCSE

In [ ]:
heatmap(dres['mcse'], peaklist, "MCSE", col, figs, "hmap_mcse")

### Noise

In [ ]:
heatmap(dres['noise'], peaklist, "Noise", col, figs, "hmap_noise")

### ESS

In [ ]:
heatmap(dres['ess'], peaklist, "ESS", col_r, figs, "hmap_ess", ".0f")